<a href="https://colab.research.google.com/github/lfernandof/lfp_signals_analysis/blob/main/state-scored-spikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import json
import scipy

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.listdir('drive/MyDrive/MSc/Rat4/2017-05-07_13-57-27')

['112_CH18.continuous',
 'cluster_quality.tsv',
 'openephys (1).CellClass.cellinfo.mat',
 'openephys.CellClass.cellinfo.mat',
 'openephys.sessionInfo.mat',
 'openephys.spikes.cellinfo.mat',
 'openephys.spkDRRD.mat',
 'frames',
 'openephys.xml',
 'openephys.spikes.waveforms.mat',
 'waveform_stability_v1.mat',
 'stability_all_cells_whole_recording_R4_D2.pdf',
 'openephys.lfp',
 '2017-05-07_13-57-27.SleepScoreLFP.LFP.mat',
 '2017-05-07_13-57-27.SleepState.states.mat',
 '2017-05-07_13-57-27.SleepStateEpisodes.states.mat',
 '2017-05-07_13-57-27.xml']

In [3]:
data_path = 'drive/MyDrive/MSc/Rat4/2017-05-07_13-57-27'

In [4]:
spikes_test = sio.loadmat(data_path+'/openephys.spikes.cellinfo.mat')

In [42]:
#UID (unit id)
uid = spikes_test['spikes'][0][0]['UID']

#times (spike times)
times = spikes_test['spikes'][0][0]['times']

#shankID, cluID, rawWaveform, maxWaveformCh follow, but I will not be using any of these

#the region each unit belongs to
region = spikes_test['spikes'][0][0]['region']

In [6]:
#Load the state scoring timestamps proper
data_states = sio.loadmat(data_path+'/2017-05-07_13-57-27.SleepState.states.mat')

awake_timestamps = data_states['SleepState']['ints'][0][0]['WAKEstate'][0][0]
nrem_timestamps = data_states['SleepState']['ints'][0][0]['NREMstate'][0][0]
rem_timestamps = data_states['SleepState']['ints'][0][0]['REMstate'][0][0]

#Not all sessions have pathological (seizure-like) intervals
try:
  pathological_timestamps = data_states['SleepState']['ints'][0][0]['events_pathologicalstate'][0][0]
except:
  pass

In [7]:
nrem_timestamps[0]

array([2632, 3010], dtype=uint16)

In [8]:
neuron1 = np.squeeze(times[0][4])

In [9]:
np.argwhere((neuron1 > 2632) * (neuron1 < 3010))

array([[11873],
       [11874],
       [11875],
       ...,
       [14449],
       [14450],
       [14451]])

In [11]:
len(neuron1[np.where((neuron1 > nrem_timestamps[0][0]) * (neuron1 < nrem_timestamps[0][1]))])

2579

In [86]:
#1. select a session (rat, day, group): DONE when loading the data and doing the state scoring timestamps
data_path = 'drive/MyDrive/MSc/Rat4/2017-05-07_13-57-27'
spikes_test = sio.loadmat(data_path+'/openephys.spikes.cellinfo.mat')

#UID (unit id)
uid = np.squeeze(spikes_test['spikes'][0][0]['UID'])

#times (spike times)
times = np.squeeze(spikes_test['spikes'][0][0]['times'])

#shankID, cluID, rawWaveform, maxWaveformCh follow, but I will not be using any of these

#the region each unit belongs to
region = np.squeeze(spikes_test['spikes'][0][0]['region'])


#Load the state scoring timestamps proper
data_states = sio.loadmat(data_path+'/2017-05-07_13-57-27.SleepState.states.mat')

awake_timestamps = data_states['SleepState']['ints'][0][0]['WAKEstate'][0][0]
nrem_timestamps = data_states['SleepState']['ints'][0][0]['NREMstate'][0][0]
rem_timestamps = data_states['SleepState']['ints'][0][0]['REMstate'][0][0]

#Not all sessions have pathological (seizure-like) intervals
try:
  pathological_timestamps = data_states['SleepState']['ints'][0][0]['events_pathologicalstate'][0][0]
except:
  pass

#1. For each neuron:
#for unit_id in uid-1: #uid is 1-indexed as per matlab, whereas python used 0-indexing
#  current_neuron = np.squeeze(times[0][unit_id])



In [93]:
uid

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44], dtype=uint8)

In [90]:
spikes_test['spikes'][0][0]['region']

array([[array(['STR'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['STR'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['STR'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        array(['PFC'], dtype='<U3'), array(['PFC'], dtype='<U3'),
        ar

In [48]:
for unit_id in np.squeeze(uid)-1: #uid is 1-indexed as per matlab, whereas python used 0-indexing
  current_spike_train = np.squeeze(times[unit_id])
  current_unit_region = np.squeeze(region)[unit_id][0]
  break

In [52]:
current_spike_train

array([2.44200000e-01, 4.11000000e-01, 8.36966667e-01, ...,
       1.21058641e+04, 1.21058885e+04, 1.21063646e+04])

array([2632.82533333, 2632.94546667, 2633.0649    , ..., 3009.19543333,
       3009.27086667, 3009.90206667])

In [ ]:
current_spike_train[np.where((current_spike_train > nrem_timestamps[0][0]) * (current_spike_train < nrem_timestamps[0][1]))]

In [57]:
for period_id,period_limits in enumerate(nrem_timestamps):
  print(period_limits[1])
  break

3010


In [69]:
day_id = 1
rat_id = 4
group_id = 2
test_df = pd.DataFrame.from_dict({'timestamp':current_spike_train,'region':current_unit_region,'day':day_id,'group':group_id,'neuron':unit_id})

In [70]:
test_df

,timestamp,region,day,group,neuron
0,0.244200,STR,1,2,0
1,0.411000,STR,1,2,0
2,0.836967,STR,1,2,0
3,0.861667,STR,1,2,0
4,1.381100,STR,1,2,0
...,...,...,...,...,...
53915,12105.057967,STR,1,2,0
53916,12105.472267,STR,1,2,0
53917,12105.864133,STR,1,2,0
53918,12105.888500,STR,1,2,0


In [60]:
def create_or_append(dataframe,current_data):
    if dataframe is not None:
        return pd.concat([dataframe,pd.DataFrame(current_data)])
    else:
        return pd.DataFrame(current_data)

In [71]:
test_df2 = None

test_df2 = create_or_append(test_df2,test_df)

In [72]:
test_df2

,timestamp,region,day,group,neuron
0,0.244200,STR,1,2,0
1,0.411000,STR,1,2,0
2,0.836967,STR,1,2,0
3,0.861667,STR,1,2,0
4,1.381100,STR,1,2,0
...,...,...,...,...,...
53915,12105.057967,STR,1,2,0
53916,12105.472267,STR,1,2,0
53917,12105.864133,STR,1,2,0
53918,12105.888500,STR,1,2,0


In [76]:
#1. Load the spikes' dataframe


In [77]:
session_data

,Unnamed: 0,timestamp,quality,region,day,group,rat,neuron,trial,time,duration
0,0,277.392548,2.0,PFC,1,1,7,0,6.0,0.110264,0.253338
1,1,277.088584,2.0,PFC,1,1,7,7,6.0,-0.193700,0.253338
2,2,276.931543,2.0,PFC,1,1,7,10,6.0,-0.350740,0.253338
3,3,276.978319,2.0,PFC,1,1,7,10,6.0,-0.303964,0.253338
4,4,276.979999,2.0,PFC,1,1,7,10,6.0,-0.302285,0.253338
...,...,...,...,...,...,...,...,...,...,...,...
1829557,29318,7600.126467,0.0,PFC,2,2,6,7,693.0,0.308167,2.417367
1829558,29319,7605.773933,0.0,PFC,2,2,6,7,694.0,-0.191033,1.954200
1829559,29320,7611.368667,0.0,PFC,2,2,6,7,695.0,-0.291467,0.188067
1829560,29321,7612.515200,0.0,PFC,2,2,6,7,696.0,0.575733,2.395200


In [118]:
def scored_state_probing(rat_id,day_id,session_data,state_type='NREMstate'):
  '''
  Find all intervals of a given type in a session and construct DataFrames with all events (spikes) comprising each period, alongside other relevant columns.

  Parameters
  ----------
  rat_id: int
          The chosen animal
  day_id: int
          The day in which the session happened
  state_type: string [default: "NREMstate"]
          The type of period which to probe. Options are: "NREMstate", "WAKEstate", "REMstate" and "events_pathologicalstate"

  Returns
  ----------
  session_staging_df: pandas DataFrame
          a pandas DataFrame with the following columns:
            - timestamp [float]: instant at which a given spike happened (in s)
            - region [string]: either 'PFC' or 'STR'
            - day [int]: either '1' or '2'
            - group [int]: only '2' for now, for we only have pre-trial spiking data from animals in group 2
            - rat [int]: each animal's ID
            - stage_type [string]: either {'WAKE','nREM','REM' or 'pathological'}
            - period_id [int]: period index for a given stage (e.g.: {stage_type = 'NREM',period_id = 0} is the first NREM period for that neuron in that session)
            - duration [float]: the length of a given period in s (e.g. {stage_type = 'NREM',period_id = 0, duration = 15} indicates that the first NREM period for that neuron in that session lasts 15 s)

  '''
  print(f'Beginning the stage segmentation of {state_type.removesuffix("state")} states from session:\n(Rat: {rat_id}, day: {day_id})')

  #Query for the desired session and find the first and last timestamp, corresponding to the beginning and end of the training protocol during this session
  training_begin = session_data.query(f'rat == {rat_id} and day == {day_id}').timestamp.min()
  training_end = session_data.query(f'rat == {rat_id} and day == {day_id}').timestamp.max()

  #Load the timestamps for each event (e.g. sleep, pathological states) as a list

  #We set up these conditionals just for rat 4, whose folders are differently named from the pattern
  if rat_id == 4 and day_id == 1:
    day_tag = '2017-05-06_09-58-35'
  elif rat_id == 4 and day_id == 2:
    day_tag = '2017-05-07_13-57-27'
  else:
    day_tag = f'Day{day_id}'

  state_scores_path = f'/content/drive/MyDrive/MSc/Rat{rat_id}/{day_tag}'

  #Load the state scoring timestamps proper
  data_states = sio.loadmat(state_scores_path+f'/{day_tag}.SleepState.states.mat')

  #Not all sessions have pathological states
  if state_type == 'events_pathologicalstate':
    try:
      event_timestamps = data_states['SleepState']['ints'][0][0][state_type][0][0]
    except:
      raise('There are no pathological (microseizure) states in this session.')
  #But all others are guaranteed
  else:
    event_timestamps = data_states['SleepState']['ints'][0][0][state_type][0][0]

  # Load the spike corresponding to this session
  data_path = f'drive/MyDrive/MSc/Rat{rat_id}/{day_tag}/'
  spikefile_tag = 'openephys'
  #exceptions to overwrite names because these folders/files are wrongly named and I do not have permission to change it
  if rat_id == 4 and day_id == 1:
    spikefile_tag = '2017-05-06_09-58-35'
  spikes_test = sio.loadmat(data_path+spikefile_tag+'.spikes.cellinfo.mat')

  #UID (unit id)
  uid = np.squeeze(spikes_test['spikes'][0][0]['UID'])

  #times (spike times)
  times = np.squeeze(spikes_test['spikes'][0][0]['times'])

  #the region each unit belongs to
  region = np.squeeze(spikes_test['spikes'][0][0]['region'])

  #shankID, cluID, rawWaveform, maxWaveformCh are also in this .mat file, but I will not be using any of these


  def create_or_append(dataframe,current_data):
    if dataframe is not None:
        return pd.concat([dataframe,pd.DataFrame(current_data)])
    else:
        return pd.DataFrame(current_data)

  session_df = None #empty variable; upon first being called a df it will be rewritten by a df; upon next calls in the loop it will be concatenated with newer inputs

  #iterate over neurons
  for unit_id in np.squeeze(uid)-1: #uid is 1-indexed as per matlab, whereas python used 0-indexing
    current_spike_train = np.squeeze(times[unit_id])
    current_unit_region = np.squeeze(region)[unit_id][0]

    #for each neuron find which of its spikes fall within each period (stage) of a given type (e.g. sleep) and add each spike as a row in a pandas DataFrame
    for period_id,period_limits in enumerate(event_timestamps):
      period_timestamps = current_spike_train[np.where((current_spike_train > period_limits[0]) * (current_spike_train < period_limits[1]))]
      period_df = pd.DataFrame.from_dict({'timestamp':period_timestamps,'region':current_unit_region,'day':day_id,'group':group_id,
                                          'neuron':unit_id,'stage':state_type,'stage_id':period_id,'stage_durtion':np.diff(period_limits)[0]})
      session_df = create_or_append(session_df,period_df)

  print('Done')
  return session_df

In [119]:
session_data = pd.read_csv('/content/drive/MyDrive/backup-mac/ufabc/entire-backup/lfp/all_channels//data/session_data-spikes.csv').copy()

rat_id = 6
day_id = 1
test_df = scored_state_probing(rat_id,day_id,session_data,state_type='NREMstate')

Beginning the stage segmentation of NREM states from session:
(Rat: 6, day: 1)
Done


There is one problem left to solve: session _rat 4, day1_ does not have a 'region' identifier in the _.mat_ file. I will have to infer each region from _shankID_, cross-referencing other sessions' files. I will do that tomorrow.

# ignore for now

In [ ]:
os.listdir('drive/MyDrive/MSc/Rat4/2017-05-07_13-57-27')

['112_CH18.continuous',
 'cluster_quality.tsv',
 'openephys (1).CellClass.cellinfo.mat',
 'openephys.CellClass.cellinfo.mat',
 'openephys.sessionInfo.mat',
 'openephys.spikes.cellinfo.mat',
 'openephys.spkDRRD.mat',
 'frames',
 'openephys.xml',
 'openephys.spikes.waveforms.mat',
 'waveform_stability_v1.mat',
 'stability_all_cells_whole_recording_R4_D2.pdf',
 'openephys.lfp',
 '2017-05-07_13-57-27.SleepScoreLFP.LFP.mat',
 '2017-05-07_13-57-27.SleepState.states.mat',
 '2017-05-07_13-57-27.SleepStateEpisodes.states.mat',
 '2017-05-07_13-57-27.xml']

In [7]:
data_path = '/content/drive/MyDrive/backup-mac/ufabc/entire-backup/lfp/all_channels/'
os.chdir(data_path)

data_states = sio.loadmat('./data/state_scoring/rat4_day2/2017-05-07_13-57-27.SleepState.states.mat')

data_episodes = sio.loadmat('./data/state_scoring/rat4_day2/2017-05-07_13-57-27.SleepStateEpisodes.states.mat')

data_scores = sio.loadmat('./data/state_scoring/rat4_day2/2017-05-07_13-57-27.SleepScoreLFP.LFP.mat')

1. get the asleep periods' durations (or # of spikes?) into a list
2. ensemble over asleep periods?

In [ ]:
#This encodes the timestamps [begin, end] for each event of a given state
#data_states['SleepState'][0][0][0][0]

#Timestamps for awake states
awake_timestamps = data_states['SleepState'][0][0][0][0][0][0]

#Timestamps for pathological states
pathological_timestamps = data_states['SleepState'][0][0][0][0][0][1]

#Timestamps for NREM states
nrem_timestamps = data_states['SleepState'][0][0][0][0][0][2]

#Timestamps for REM states
rem_timestamps = data_states['SleepState'][0][0][0][0][0][3]

#'''caution: I have figured which timestamps belong to which state due to the dtypes assigned at the level of "data_states['SleepState'][0][0][0][0]", but it might be a mistake of mine

In [ ]:
times[0][0] #primeiro neuronio

array([[2.44200000e-01],
       [4.11000000e-01],
       [8.36966667e-01],
       ...,
       [1.21058641e+04],
       [1.21058885e+04],
       [1.21063646e+04]])

(array([], dtype=int64),)

In [ ]:
nrem_timestamps[0][0] < test < nrem_timestamps[0][1]

True

In [2]:
rat_id = 4
day_id = 1

#4. Load the timestamps for each event (e.g. sleep, pathological states) as a list

###we set up these conditionals just for rat 4, whose folders are differently named from the pattern
if rat_id == 4 and day_id == 1:
  day_tag = '2017-05-06_09-58-35'
elif rat_id == 4 and day_id == 2:
  day_tag = '2017-05-07_13-57-27'
else:
  day_tag = f'Day{day_id}'

state_scores_path = f'/content/drive/MyDrive/MSc/Rat{rat_id}/{day_tag}'

#Load the state scoring timestamps proper
data_states = sio.loadmat(state_scores_path+f'/{day_tag}.SleepState.states.mat')

awake_timestamps = data_states['SleepState']['ints'][0][0]['WAKEstate'][0][0]
nrem_timestamps = data_states['SleepState']['ints'][0][0]['NREMstate'][0][0]
rem_timestamps = data_states['SleepState']['ints'][0][0]['REMstate'][0][0]

#Not all sessions have pathological (seizure-like) intervals
try:
  pathological_timestamps = data_states['SleepState']['ints'][0][0]['events_pathologicalstate'][0][0]
except:
  pass

In [11]:
spikes_test = sio.loadmat(state_scores_path+f'/{day_tag}.openephys.spikes.cellinfo.mat')

FileNotFoundError: ignored

In [10]:
os.listdir(state_scores_path)

['settings.xml',
 '112_CH12.continuous',
 '112_CH2.continuous',
 '112_CH22.continuous',
 '112_CH31.continuous',
 '112_CH25.continuous',
 '112_CH3.continuous',
 '112_CH27.continuous',
 '112_CH21.continuous',
 '112_CH24.continuous',
 '112_CH32.continuous',
 '112_CH16.continuous',
 '112_CH14.continuous',
 '112_CH10.continuous',
 '112_CH19.continuous',
 '112_CH30.continuous',
 '112_CH4.continuous',
 '112_CH9.continuous',
 '112_CH5.continuous',
 '112_CH26.continuous',
 '112_CH15.continuous',
 '112_CH11.continuous',
 '112_CH28.continuous',
 '112_CH1.continuous',
 '112_CH20.continuous',
 '112_CH7.continuous',
 '112_CH18.continuous',
 '112_CH23.continuous',
 '112_AUX1.continuous',
 '112_CH8.continuous',
 '112_CH29.continuous',
 '112_CH13.continuous',
 'Continuous_Data.openephys',
 'messages.events',
 '112_AUX2.continuous',
 '112_CH17.continuous',
 '112_AUX3.continuous',
 '112_CH6.continuous',
 'cluster_quality.tsv',
 'frames',
 'Kilosort2_2021-06-24_230901',
 '2017-05-06_09-58-35.spikes.cellin